In [2]:
!pip install tensorflow-gpu

     |████████████████████████████████| 394.7MB 44kB/s 


In [3]:
import tensorflow as tf
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K
from tensorflow.keras.applications.inception_v3 import InceptionV3


In [6]:
model = InceptionV3(include_top=False, weights='imagenet')

87916544/87910968 [==============================] - 1s 0us/step


In [ ]:
model.summary()

In [8]:
import scipy
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import inception_v3
def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)


def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    scipy.misc.imsave(fname, pil_img)


def preprocess_image(image_path):
    # Util function to open, resize and format pictures
    # into appropriate tensors.
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img


def deprocess_image(x):
    # Util function to convert a tensor into a valid image.
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [113]:
import numpy as np

# Playing with these hyperparameters will also allow you to achieve new effects

step = 0.01  # Gradient ascent step size
num_octave = 3  # Number of scales at which to run gradient ascent
octave_scale = 1.4  # Size ratio between scales
iterations = 20  # Number of ascent steps per scale

# If our loss gets larger than 10,
# we will interrupt the gradient ascent process, to avoid ugly artifacts
max_loss = 10.

# Fill this to the path to the image you want to use
base_image_path = '/content/1.jpg'

# Load the image into a Numpy array
img = preprocess_image(base_image_path)

# We prepare a list of shape tuples
# defining the different scales at which we will run gradient ascent
original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)

# Reverse list of shapes, so that they are in increasing order
successive_shapes = successive_shapes[::-1]

# Resize the Numpy array of the image to our smallest scale
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])



In [11]:
layer_weights = [0.2, 3.0, 2.0, 1.5]
layer_list = ['mixed2', 'mixed3', 'mixed4', 'mixed5']

### **test one step gradient ascent**


**make a input tensor: tf variablr initialized by shrunk_original_img**

In [87]:
shrunk_original_img = tf.Variable(initial_value = shrunk_original_img)

build a new model to get the output of those layers

In [99]:
dream_model = tf.keras.Model(model.input, [model.get_layer(layer).output for layer in layer_list])

**build grad tape**

In [100]:
with tf.GradientTape(persistent=True) as tape:
    loss = tf.constant(0.0)
    tape.watch(shrunk_original_img)
    outputs = dream_model(shrunk_original_img)
    for i, o in enumerate(outputs):
        # print(type(o))   # is it possible that the reason of ' the tape is broken '  is that the type of those values is 'np.darray'?
        scaling = K.prod(K.cast(K.shape(o), 'float32'))
        loss += layer_weights[i] * K.sum(K.square(o[:, 2: -2, 2: -2, :])) / scaling

In [101]:
loss

<tf.Tensor: shape=(), dtype=float32, numpy=1.6337845>

**compute grads**

In [102]:
grads = tape.gradient(loss, shrunk_original_img)

In [103]:
print(grads)

tf.Tensor(
[[[[-2.3837952e-06  1.5734507e-06 -7.8990792e-07]
   [-3.4992504e-06  4.7754834e-06 -1.0679682e-06]
   [ 3.6564850e-06  8.3603827e-06 -1.5219522e-06]
   ...
   [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
   [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
   [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]]

  [[-1.3711244e-06  3.2263338e-06 -1.4987492e-06]
   [ 8.2639480e-07  9.3457211e-06 -1.1410708e-06]
   [ 2.7219692e-06  7.6256574e-06 -2.7848919e-06]
   ...
   [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
   [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
   [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]]

  [[ 5.4563588e-06  4.5083175e-06 -1.7950000e-06]
   [ 5.7759848e-06  5.7993770e-06 -4.2263532e-06]
   [ 4.7469598e-06  3.3259362e-06 -2.7874096e-06]
   ...
   [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
   [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
   [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]]

  ...

  [[-1.5850527e-07  4.3354154e-07 -1.9741981e-07]
   [-

### **deep dreaming**

In [104]:
dream_model = tf.keras.Model(model.input, [model.get_layer(layer).output for layer in layer_list])

In [105]:
def fetch_loss_and_grads(img):
    with tf.GradientTape() as tape:
        loss = 0
        img = tf.Variable(img)
        outputs = dream_model(img)
        for i, o in enumerate(outputs):
            # We avoid border artifacts by only involving non-border pixels in the loss.
            scaling = K.prod(K.cast(K.shape(o), 'float32'))
            loss += layer_weights[i] * K.sum(K.square(o[:, 2: -2, 2: -2, :])) / scaling
    
    grads = tape.gradient(loss, img)[0]
    grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)
    outputs = [loss, grads]
    return outputs

In [106]:
def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads(x)
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Loss value at', i, ':', loss_value)
        x += step * grad_values
    return x

In [114]:
for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_img(img, shape)
    tensor_img = tf.Variable(img)
    img = gradient_ascent(img, iterations=iterations, step=step, max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img

    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    # save_img(img, fname='dream_at_scale_' + str(shape) + '.png')
#

Processing image shape (244, 434)
...Loss value at 0 : tf.Tensor(1.6337845, shape=(), dtype=float32)
...Loss value at 1 : tf.Tensor(2.0448835, shape=(), dtype=float32)
...Loss value at 2 : tf.Tensor(2.6504216, shape=(), dtype=float32)
...Loss value at 3 : tf.Tensor(3.2228754, shape=(), dtype=float32)
...Loss value at 4 : tf.Tensor(3.865238, shape=(), dtype=float32)
...Loss value at 5 : tf.Tensor(4.5367727, shape=(), dtype=float32)
...Loss value at 6 : tf.Tensor(5.242536, shape=(), dtype=float32)
...Loss value at 7 : tf.Tensor(5.885832, shape=(), dtype=float32)
...Loss value at 8 : tf.Tensor(6.4866176, shape=(), dtype=float32)
...Loss value at 9 : tf.Tensor(7.1043987, shape=(), dtype=float32)
...Loss value at 10 : tf.Tensor(7.674169, shape=(), dtype=float32)
...Loss value at 11 : tf.Tensor(8.289647, shape=(), dtype=float32)
...Loss value at 12 : tf.Tensor(8.802347, shape=(), dtype=float32)
...Loss value at 13 : tf.Tensor(9.415799, shape=(), dtype=float32)
...Loss value at 14 : tf.Tensor

In [115]:
pil_img = deprocess_image(np.copy(img))

In [116]:
pil_img.shape

(480, 852, 3)

In [117]:
from PIL import Image
img = Image.fromarray(pil_img, 'RGB')
img.save('my.png')
img.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')